## Training with a Neural Network
This notebook contains the training of a Keras Sequential model on insider data, which attempts to predict the maximum 90-day percentage gain of a ticker whose insider(s) made a trade. We use a single dense hidden layer and a dropout layer, and we use the Keras Tuner to choose an optimal learning rate and number of hidden units.

In [1]:
%load_ext autoreload
%autoreload 1
%aimport my_functions

import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, confusion_matrix 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from operator import itemgetter

from my_functions import *

In [2]:
DAYS_TO_LOOK_BACK = 6  # used for calculating volume volatility and recent-trade counts

train_and_cv = prepareForModel(pd.read_csv('data/training_and_cv_data.csv'))

startDate = min(train_and_cv.FilingDate) + dt.timedelta(days=DAYS_TO_LOOK_BACK)
endDate = max(train_and_cv.FilingDate)
splitDate = startDate + dt.timedelta(days=int(0.9*(endDate-startDate).days))

train_XY, train_X, train_Y = returnXandY(train_and_cv, 
                                         dt.date.isoformat(startDate), 
                                         dt.date.isoformat(splitDate))

cv_XY, cv_X, cv_Y = returnXandY(train_and_cv, 
                                dt.date.isoformat(splitDate+dt.timedelta(days=1)), dt.date.isoformat(
                                    endDate))

In [3]:
print(f'train shape: {train_X.shape}')
print(f'cv shape: {cv_X.shape}')

train shape: (4670, 16)
cv shape: (521, 16)


In [4]:
'''
Perform standard feature normalization.
'''
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
cv_X_scaled = scaler.fit_transform(cv_X)

Here is a StackExchange answer that provides a starting point for deciding on the number of hidden units to include: https://stats.stackexchange.com/a/136542

### Below is a computation to obtain the absolute maximum number of hidden units:

In [5]:
Ns = train_X_scaled.shape[0]  # training examples
No = 1                           # output neurons
Ni = train_X_scaled.shape[1]  # input neurons
alpha = 2                        # scale factor

Nh = Ns / (alpha*(Ni + No))      # maximum hidden neurons

print(f'An upper bound for number of hidden units: {int(Nh)}')

An upper bound for number of hidden units: 137


This seems like *quite* a lot; we certainly don't need this many hidden units! 

### However, we can remove some of the guesswork by using Keras Tuner. 
With this tool, we can search the parameter space and also determine an optimal number of hidden units.

We have 16 inputs, so let's opt for one Dense and one Dropout hidden layer.

In [6]:
from tensorflow.python.ops import math_ops, numpy_ops
numpy_ops.np_config.enable_numpy_behavior()

def asymmetric_loss(wgt):
    '''This is our custom objective loss function that favors either underestimates (wgt > 1)
    or overestimates (0 < wgt < 1).'''
    def custom_loss(y_true, y_pred):
        diff = wgt*math_ops.squared_difference(y_pred, y_true)*(y_true < y_pred).astype(float) + \
                math_ops.squared_difference(y_pred, y_true)*(y_true >= y_pred).astype(float)
        
        loss = tf.math.sqrt(tf.reduce_mean(diff, axis=-1))

        return loss
    return custom_loss

In [8]:
import keras_tuner as kt

tf.random.set_seed(40)

def model_builder(numFeatures):
    def builder(tuner):
        numUnits = tuner.Int('units', min_value=4, max_value=32, step=4)
        learningRate = tuner.Choice('learningRate', values=[1e-2, 1e-3, 1e-4])
        UNDERESTIMATE_BIAS = tuner.Choice('UNDERESTIMATE_BIAS', values=[1, 2, 5, 10, 20, 50])
        
        model = Sequential(
            [               
                Input(shape=(numFeatures,)),
                Dense(units=numUnits, activation='relu', name='dense_1'),
                Dropout(0.1),
                Dense(units=1, activation='linear', name='dense_2')
            ], name = 'nn_model' 
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learningRate),
            loss=asymmetric_loss(UNDERESTIMATE_BIAS),
            metrics=asymmetric_loss(UNDERESTIMATE_BIAS)
        )

        return model
    return builder

Hyperband is an algorithm that searches the hyperparameter space with respect to which we want to minimize the evaluation metric, i.e. the custom asymmetric loss. 

We use EarlyStopping to halt training early if there is no loss improvement in the 10 most recent epochs.

In [10]:
tuner = kt.Hyperband(
    model_builder(train_X_scaled.shape[1]),
    objective=asymmetric_loss(UNDERESTIMATE_BIAS),
    max_epochs=30,
    overwrite=True,
    directory='tuner logs',
    project_name='asymmetric_MSE'
)

stopEarly = tf.keras.callbacks.EarlyStopping(patience=10)

tuner.search(
    train_X_scaled, train_Y, 
    epochs=30,
    validation_data=(cv_X_scaled, cv_Y),
    callbacks=[stopEarly]
)

best_hparams = tuner.get_best_hyperparameters()[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
4                 |?                 |units
0.0001            |?                 |learningRate
50                |?                 |UNDERESTIMATE_BIAS
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
3                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2
146/146 [==============================] - 1s 2ms/step - loss: 428.3163 - custom_loss: 428.3163 - val_loss: 425.7496 - val_custom_loss: 425.7496
Epoch 2/2
146/146 [==============================] - 0s 1ms/step - loss: 427.6000 - custom_loss: 427.6000 - val_loss: 424.8866 - val_custom_loss: 424.8866


TypeError: Expected the return value of HyperModel.fit() to be a single float when `objective` is left unspecified. Recevied return value: <keras.callbacks.History object at 0x0000025BB23955E0> of type <class 'keras.callbacks.History'>.

In [103]:
print(f'These are the best hyperparameter values: \n {best_hparams.values}')
nn_model = tuner.hypermodel.build(best_hparams)

These are the best hyperparameter values: 
 {'units': 32, 'learningRate': 0.01, 'UNDERESTIMATE_BIAS': 1, 'tuner/epochs': 10, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}


We aren't surprised to see that loss is minimized for UNDERESTIMATE_BIAS=1, i.e. a symmetric mean squared error, because increasing the bias directly increases the loss accumulated from each overestimate. I initially thought about only allowing values greater than 1, but as will be seen below, we have another, more pressing problem!

In [102]:
train_Y_preds = nn_model.predict(train_X_scaled)
cv_Y_preds = nn_model.predict(cv_X_scaled)

print(f'\ncv MSE: {mean_squared_error(cv_Y, cv_Y_preds)}\n')

dfcv = pd.DataFrame(data={'cvPreds': np.reshape(cv_Y_preds, (cv_Y_preds.size,)), 'cvVals': cv_Y})

17/17 [==============================] - 0s 742us/step

cv MSE: 784.2179197510907



Note that this cv MSE is ~100 points higher than that produced by the XGBoost model.

In [105]:
n = 5
nlargest = dfcv.nlargest(n, ['cvPreds'])
nsmallest = dfcv.nsmallest(n, ['cvPreds'])
print(f'The {n} largest predictions and their values: \n{nlargest}\n')
print(f'The {n} smallest predictions and their values: \n{nsmallest}')

The 5 largest predictions and their values: 
        cvPreds      cvVals
5153  10.747974   10.305895
5156  10.728843   10.305895
5157  10.154403   10.305895
5245   1.661836  225.657188
5246   1.661831  225.657188

The 5 smallest predictions and their values: 
       cvPreds     cvVals
5320 -2.920911   6.650825
5101 -2.514010  63.048783
5283 -2.264927  -2.651003
5177 -2.098195  15.254239
5435 -2.057703  28.778254


### Here, we've reached a bit of a standstill in the modeling process.
Recall from above that these predictions are generated with UNDERESTIMATE_BIAS=1 -- there is no bias towards underestimates in our objective function! Yet all but 3 cross-validation estimates are well under 2%, and all but 3 estimates are in the interval \[-3, 2\]%.

This is puzzling, and I need to think more about why this would be. These predictions are kind of useless -- using them to create an investment strategy probably wouldn't result in us losing money, but we also likely wouldn't gain much, especially relative to the S&P500 gaining 8% in this time period!

However, the XGBoost model seems to be working better, so let's implement a strategy with that model in strategy_simulation.